User Engagement analysis

In [2]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
os.chdir('..')
from db_connection.connection import PostgresConnection

from src.utils import missing_values_table, convert_bytes_to_megabytes




Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [3]:
db = PostgresConnection(dbname='postgres', user='postgres', password='ocho')
db.connect()
# Example query
query = "SELECT * FROM xdr_data"
result = db.execute_query(query)

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result, columns=[desc[0] for desc in db.cursor.description])
print(df.head())  # Display the first few rows of the DataFrame

# Close the connection when done
db.close_connection()


Connected to PostgreSQL database!
      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...         

In [13]:
def aggregate_engagement_metrics(df):
    return df.groupby('MSISDN').agg({
        'sessions_frequency': 'sum',
        'session_duration': 'sum',
        'session_total_traffic': 'sum'
    })

In [5]:
def report_top_customers(customer_engagement):
    top_10_frequency = customer_engagement['sessions_frequency'].nlargest(10)
    top_10_duration = customer_engagement['session_duration'].nlargest(10)
    top_10_traffic = customer_engagement['session_total_traffic'].nlargest(10)
    return top_10_frequency, top_10_duration, top_10_traffic


In [14]:
def normalize_data(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)


In [7]:
def run_kmeans_clustering(data, k=3):
    kmeans = KMeans(n_clusters=k, random_state=42)
    data['cluster'] = kmeans.fit_predict(data)
    return data

In [8]:
def compute_cluster_metrics(customer_engagement):
    return customer_engagement.groupby('cluster').agg({
        'sessions_frequency': ['min', 'max', 'mean', 'sum'],
        'session_duration': ['min', 'max', 'mean', 'sum'],
        'session_total_traffic': ['min', 'max', 'mean', 'sum']
    })

In [15]:
def aggregate_user_app_traffic(data):
    return data.groupby(['MSISDN', 'application']).agg({
        'total_traffic': 'sum'
    })

In [16]:
def plot_top_applications(user_app_traffic):
    top_3_applications = user_app_traffic.groupby('application')['total_traffic'].sum().nlargest(3)
    top_3_applications.plot(kind='bar', xlabel='Application', ylabel='Total Traffic', title='Top 3 Most Used Applications')
    plt.show()

In [17]:
def elbow_method_for_optimal_k(data):
    distortions = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, random_state=42)
        kmeans.fit(data)
        distortions.append(kmeans.inertia_)

    plt.plot(range(1, 11), distortions, marker='o')
    plt.xlabel('Number of Clusters')
    plt.ylabel('Distortion')
    plt.title('Elbow Method for Optimal K')
    plt.show()


In [19]:

# Main function to execute the tasks
def main():

    customer_engagement = aggregate_engagement_metrics(df)

    top_10_frequency, top_10_duration, top_10_traffic = report_top_customers(customer_engagement)

    normalized_data = normalize_data(customer_engagement)

    clustered_data = run_kmeans_clustering(normalized_data)

    cluster_metrics = compute_cluster_metrics(clustered_data)

    user_app_traffic = aggregate_user_app_traffic(data)

    plot_top_applications(user_app_traffic)

    elbow_method_for_optimal_k(normalized_data)

if __name__ == '__main__':
    main()

KeyError: 'MSISDN'